In [4]:
import numpy as np
import random

num_entities = 10
num_validators = 500000
num_slots = 300000
fixed_reward = 0.025
sigma_mev_reward = 1
split_threshold = 32
num_builders = 10
threshold_percentage = 0.2

# Define the mean MEV rewards for each builder
builder_mev_mean_rewards = [0.5] + [0.3]*2 + [0.1]*(num_builders-3)

# Create validator_entity_mapping, builder_entity_mapping and initialize rewards
validator_entity_mapping = [i % num_entities for i in range(num_validators)]
builder_entity_mapping = [i % num_entities for i in range(num_builders)]
validator_rewards = np.zeros(num_validators)
entity_rewards = np.zeros(num_entities)
builder_profits = np.zeros(num_builders)

# Create lists of validators per entity
validators_per_entity = [[] for _ in range(num_entities)]
for validator_idx, entity in enumerate(validator_entity_mapping):
    validators_per_entity[entity].append(validator_idx)

# Run the simulation
for _ in range(num_slots):
    # Select a random validator
    chosen_validator = random.randrange(num_validators)

    # Calculate block values for each builder
    block_values = [np.random.lognormal(mean=np.log(mean_reward), sigma=sigma_mev_reward) for mean_reward in builder_mev_mean_rewards]

    # Choose the highest block value
    chosen_builder = np.argmax(block_values)
    mev_reward = block_values[chosen_builder]

    # Check if the validator's entity's builder has a block value within the threshold percentage
    validator_entity = validator_entity_mapping[chosen_validator]
    entity_builder = builder_entity_mapping.index(validator_entity)
    entity_builder_value = block_values[entity_builder]

    if entity_builder_value >= (1 - threshold_percentage) * mev_reward:
        chosen_builder = entity_builder
        mev_reward = entity_builder_value

    # Share the reward between the validator and the builder
    validator_share = mev_reward * 0.95
    builder_share = mev_reward * 0.05
    builder_profits[chosen_builder] += builder_share

    # Assign rewards
    block_reward = fixed_reward
    total_reward = block_reward + mev_reward

    validator_rewards[chosen_validator] += total_reward
    entity_rewards[validator_entity_mapping[chosen_validator]] += total_reward

    # Check if we need to spawn a new validator for any entity
    for entity_idx in range(num_entities):
        if entity_rewards[entity_idx] >= split_threshold:
            # Spawn a new validator for this entity
            num_validators += 1
            validator_entity_mapping.append(entity_idx)
            validator_rewards = np.append(validator_rewards, 0)
            validators_per_entity[entity_idx].append(num_validators - 1)

            # Deduct the rewards used to spawn the new validator from the existing validators
            to_deduct = split_threshold
            for validator_idx in validators_per_entity[entity_idx]:
                if to_deduct > 0:
                    deduct_amount = min(validator_rewards[validator_idx], to_deduct)
                    validator_rewards[validator_idx] -= deduct_amount
                    entity_rewards[entity_idx] -= deduct_amount
                    to_deduct -= deduct_amount

# Print the top and bottom 20 validator stakes
sorted_indices = np.argsort(validator_rewards)
print("Top 20 validator stakes:")
for idx in sorted_indices[-20:]:
    print(f"Validator {idx}: {validator_rewards[idx]}")

print("Bottom 20 validator stakes:")
for idx in sorted_indices[:20]:
    print(f"Validator {idx}: {validator_rewards[idx]}")

# Count validators per entity
validators_count_per_entity = [len(validators) for validators in validators_per_entity]

# Calculate the total value each entity controls
entity_values = np.array(validators_count_per_entity) * 32 + entity_rewards

# Print the results
print("Validators per entity:")
for i in range(num_entities):
    print(f"Entity {i}: {validators_count_per_entity[i]}")

print("Total value each entity controls:")
for i in range(num_entities):
    print(f"Entity {i}: {entity_values[i]}")

# Rank entities by their total value
sorted_entity_indices = np.argsort(entity_values)[::-1]
print("Entities ranked by total value:")
for rank, idx in enumerate(sorted_entity_indices, start=1):
    print(f"Rank {rank}: Entity {idx} with total value {entity_values[idx]}")

# Rank builders by their total profits
sorted_builder_indices = np.argsort(builder_profits)[::-1]
print("Builders ranked by total profits:")
for rank, idx in enumerate(sorted_builder_indices, start=1):
    print(f"Rank {rank}: Builder {idx} with total profits {builder_profits[idx]}")


Top 20 validator stakes:
Validator 191717: 1.934715339502708
Validator 178653: 2.0552077178227415
Validator 189256: 2.056973649971191
Validator 15330: 2.1619547258418885
Validator 438609: 2.3064816597088678
Validator 314361: 2.4517264897490634
Validator 154450: 2.480784146848886
Validator 121017: 2.5042046959570783
Validator 144815: 2.6652274362673736
Validator 382137: 2.690344398135019
Validator 101946: 2.789176301096702
Validator 367631: 2.9273399017256003
Validator 191819: 2.935085891891608
Validator 26123: 3.019902627225665
Validator 246610: 3.04004298182666
Validator 193749: 3.134333658065925
Validator 305336: 4.535767287353832
Validator 293981: 5.347896342753917
Validator 200822: 5.546575332109068
Validator 353007: 6.692904680811858
Bottom 20 validator stakes:
Validator 0: 0.0
Validator 341249: 0.0
Validator 341248: 0.0
Validator 341247: 0.0
Validator 341246: 0.0
Validator 341245: 0.0
Validator 341244: 0.0
Validator 341243: 0.0
Validator 341242: 0.0
Validator 341241: 0.0
Validato

In [6]:
# Calculate associated builder rewards for each entity
entity_builder_rewards = np.zeros(num_entities)
for builder_idx, builder_reward in enumerate(builder_profits):
    entity_builder_rewards[builder_entity_mapping[builder_idx]] += builder_reward

# Calculate the staked ETH for each entity
staked_eth_per_entity = np.array(validators_count_per_entity) * 32

# Calculate the total rewards for each entity
total_entity_rewards = entity_rewards + entity_builder_rewards + staked_eth_per_entity

# Rank entities by total rewards
sorted_total_entity_indices = np.argsort(total_entity_rewards)[::-1]

# Print the results
print("Entities ranked by total rewards including staked ETH:")
for rank, idx in enumerate(sorted_total_entity_indices, start=1):
    print(f"Rank {rank}: Entity {idx} with validator rewards {entity_rewards[idx]}, builder rewards {entity_builder_rewards[idx]}, staked ETH {staked_eth_per_entity[idx]}, and total rewards {total_entity_rewards[idx]}")


Entities ranked by total rewards including staked ETH:
Rank 1: Entity 0 with validator rewards 25.096476609786308, builder rewards 8824.571007242634, staked ETH 1637760, and total rewards 1646609.6674838525
Rank 2: Entity 1 with validator rewards 22.87444274204683, builder rewards 3613.3514934243594, staked ETH 1638080, and total rewards 1641716.2259361665
Rank 3: Entity 2 with validator rewards 21.922078711335807, builder rewards 3670.153124415641, staked ETH 1638016, and total rewards 1641708.075203127
Rank 4: Entity 7 with validator rewards 30.35795093312248, builder rewards 361.7721341378936, staked ETH 1638464, and total rewards 1638856.130085071
Rank 5: Entity 9 with validator rewards 18.87019959017051, builder rewards 373.1864974873898, staked ETH 1638432, and total rewards 1638824.0566970776
Rank 6: Entity 3 with validator rewards 9.307710155042924, builder rewards 378.4712108037623, staked ETH 1638432, and total rewards 1638819.7789209587
Rank 7: Entity 8 with validator reward